In [ ]:
from sklearn.datasets import load_files
import numpy as np

train_dir = '../input/fruits/Fruits_1/Training'
test_dir = '../input/fruits/Fruits_1/Test'

def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    target_labels = np.array(data['target_names'])
    return files,targets,target_labels
    
x_train, y_train,target_labels = load_dataset(train_dir)
x_test, y_test,_ = load_dataset(test_dir)
print('Loading complete!')

print('Training set size : ' , x_train.shape[0])
print('Testing set size : ', x_test.shape[0])

In [ ]:
target_labels

In [ ]:
no_of_classes = len(np.unique(y_train))
no_of_classes

In [ ]:
x_train.shape

In [ ]:
x_train

In [ ]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train,no_of_classes)
y_test = np_utils.to_categorical(y_test,no_of_classes)
y_train[0]

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import keras
import numpy as np # linear algebra
import keras.backend as K 
import time as ti 
import cv2
import os
import glob # for including images
import scipy.io as sio
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from tensorflow.python.keras.layers import Conv2D, DepthwiseConv2D, MaxPooling2D, AveragePooling2D  
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import RMSprop, SGD, Adadelta, Adam 
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
len(x_train)

In [ ]:
x_test,x_valid =x_test[3500:],x_test[:3500]
y_test,y_valid = y_test[3500:],y_test[:3500]
print('Vaildation X : ',x_valid.shape)
print('Vaildation y :',y_valid.shape)
print('Test X : ',x_test.shape)
print('Test y : ',y_test.shape)

In [ ]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img

def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        # Convert to Numpy Array
        images_as_array.append(img_to_array(load_img(file)))
    return images_as_array

x_train = np.array(convert_image_to_array(x_train))
print('Training set shape : ',x_train.shape)

x_valid = np.array(convert_image_to_array(x_valid))
print('Validation set shape : ',x_valid.shape)

x_test = np.array(convert_image_to_array(x_test))
print('Test set shape : ',x_test.shape)

print('1st training image shape ',x_train[0].shape)

In [ ]:
x_train/=255
x_valid/=255
x_test/=255

In [ ]:
# model = Sequential()
# nf1=20; nf2=30; nf3=10; 
# pad='same'
    
# # First conv layer
# model.add(Conv2D(filters=nf1,padding=pad, kernel_size=(3,3),input_shape=(100,100,3), activation='relu',))
# model.add(MaxPooling2D(pool_size=(4, 4),strides=(2,2),padding=pad))
    
# # Second conv layer
# model.add(Conv2D(filters=nf2,padding=pad, kernel_size=(3,3), activation='relu',))
# model.add(MaxPooling2D(pool_size=(4, 4),strides=(2,2),padding=pad))
    
# # Third conv layer 
# model.add(Conv2D(filters=nf3,padding=pad, kernel_size=(3,3), activation='relu',))
# model.add(MaxPooling2D(pool_size=(4, 4),strides=(2,2),padding=pad))
 
# model.add(GlobalAveragePooling2D())
# # model.add(Flatten())
    
# # Output with 131 neurons ( 131 classes )
# model.add(Dense(41, activation='softmax'))
    
# # model.compile(loss='categorical_crossentropy', 
# #               optimizer='adam',
# #               metrics=['accuracy',recall])

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import numpy as np

tf.compat.v1.set_random_seed(
    0
)
np.random.seed(0)

model=Sequential()
model.add(Conv2D(filters = 16, kernel_size = 2,input_shape=(100,100,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 32,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 64,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 128,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 256,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(150))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(41,activation = 'softmax'))
model.summary()

In [ ]:
def recall(y_true, y_pred):
     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
     possible_positives= K.sum(K.round(K.clip(y_true, 0, 1)))
     recall = true_positives / (possible_positives + K.epsilon())
     return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
model.compile(loss='categorical_crossentropy',
              optimizer='nadam',
              metrics=['accuracy',recall,precision])
print('Compiled!')

In [ ]:
from keras.models import Sequential
import tensorflow as tf
import numpy as np
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K
# batch_size = 100
# input_shape=(1001,100,3)
# epochs=30

tf.compat.v1.set_random_seed(0)
np.random.seed(0)
checkpointer = ModelCheckpoint(filepath = 'cnn_from_scratch_fruits.hdf5', verbose = 2, save_best_only = True,mode='max',monitor='val_accuracy')


train_image_generator = ImageDataGenerator(rotation_range=45,width_shift_range=.15,height_shift_range=.15,horizontal_flip=True, zoom_range=0.3)
train_data_gen = train_image_generator.flow(x_train,y_train)





# history=model.fit(train_data_gen,
#           steps_per_epoch=500, epochs=10,validation_data=(x_valid,y_valid),
#         callbacks = [checkpointer],
#                     verbose=2,shuffle=True)

history=model.fit(train_data_gen,
                       steps_per_epoch=500,
                       epochs=10,
                       verbose=1,
                       validation_data=(x_valid,y_valid),
                       callbacks=[checkpointer]
                      )

In [ ]:
dict_idx_fruit = {idx: label for idx, label in enumerate(target_labels)}
print(dict_idx_fruit)

In [ ]:
num_categories = len(np.unique(y_train))
num_categories

In [ ]:
model.load_weights('cnn_from_scratch_fruits.hdf5')

In [ ]:
# test_image_generator.flow(x_test,y_test)
score = model.evaluate(x_test,y_test)
print('\n', 'Test accuracy:', score[1])

In [ ]:
preds=np.round(model.predict(x_test),0)
print("Rounded test_labels",preds)

In [ ]:
#Let's visualize the first 10 training images!
import matplotlib.pyplot as plt

fig = plt.figure(figsize =(30,5))
for i in range(10):
    ax = fig.add_subplot(2,5,i+1,xticks=[],yticks=[])
    ax.imshow(np.squeeze(x_train[i]))
# Yummy fruits ;)

In [ ]:
# Let's visualize test prediction.

y_pred = model.predict(x_test)

# plot a random sample of test images, their predicted labels, and ground truth
fig = plt.figure(figsize=(16, 9))
for i, idx in enumerate(np.random.choice(x_test.shape[0], size=16, replace=False)):
    ax = fig.add_subplot(4, 4, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[idx]))
    pred_idx = np.argmax(y_pred[idx])
    true_idx = np.argmax(y_test[idx])
    ax.set_title("{} ({})".format(target_labels[pred_idx], target_labels[true_idx]),
                 color=("green" if pred_idx == true_idx else "red"))

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(figsize=(20,10))
   
 # summarize history for accuracy  
   
plt.subplot(121)  
plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'],color='red')  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'validation'], loc='upper left')  
   
 # summarize history for loss  
   
plt.subplot(122)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'],color='red')  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'validation'], loc='upper left')

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(
    y_test.argmax(axis=1), preds.argmax(axis=1))
cm

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
classification=metrics.classification_report(y_test,preds)
print(classification)

In [ ]:
cm.shape

In [ ]:


import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm = pd.DataFrame(cm, index = [i for i in range(41)],
                  columns = [i for i in range(41)])
plt.figure(figsize = (35,35))
sn.heatmap(df_cm, annot=True,cmap="YlGnBu")

In [ ]:
y_test.shape

In [ ]:
# for layer in model.layers:
#     if 'conv2d' in layer.name:
#         weights, bias= layer.get_weights()
#         print(layer.name, filters.shape)
        
#         #normalize filter values between  0 and 1 for visualization
#         f_min, f_max = weights.min(), weights.max()
#         filters = (weights - f_min) / (f_max - f_min)  
#         print(filters.shape[3])
#         filter_cnt=1
        
#         #plotting all the filters
#         for i in range(filters.shape[5]):
#             #get the filters
#             filt=filters[:,:,:, i]
#             #plotting each of the channel, color image RGB channels
#             for j in range(filters.shape[0]):
#                 ax= plt.subplot(filters.shape[3], filters.shape[0], filter_cnt  )
#                 ax.set_xticks([])
#                 ax.set_yticks([])
#                 plt.imshow(filt[:,:, j])
#                 filter_cnt+=1
#         plt.show()

In [ ]:
from sklearn import metrics
metrics.roc_auc_score(y_test, preds,multi_class="ovr",average='weighted')